# Tutorial HANNAH Framework 
## AutoML School
### Hannover 2024 

### Create Search Space
First we create a simple search space. This search space description is written to a file such that the `hydra` config framework
can refer to it and instantiate the search space. 

The search space consists of a variable number of `conv_relu` blocks with a `linear` layer at the end. 



In [1]:
%%writefile space.py
from hannah.nas.functional_operators.op import ChoiceOp, Tensor, scope
from hannah.nas.functional_operators.operators import AdaptiveAvgPooling, Conv2d, Linear, Relu
from hannah.nas.functional_operators.op import search_space
from hannah.nas.parameters.parameters import CategoricalParameter, IntScalarParameter


@scope
def conv_relu(input, kernel_size, out_channels, stride):
    in_channels = input.shape()[1]
    weight = Tensor(name='weight',
                    shape=(out_channels, in_channels, kernel_size, kernel_size),
                    axis=('O', 'I', 'kH', 'kW'),
                    grad=True)

    conv = Conv2d(stride=stride)(input, weight)
    relu = Relu()(conv)
    return relu


def adaptive_avg_pooling(input):
    return AdaptiveAvgPooling()(input)


def dynamic_depth(*exits, switch):
    return ChoiceOp(*exits, switch=switch)()


def linear(input, out_features):
    input_shape = input.shape()
    in_features = input_shape[1] * input_shape[2] * input_shape[3]
    weight = Tensor(name='weight',
                    shape=(in_features, out_features),
                    axis=('in_features', 'out_features'),
                    grad=True)

    out = Linear()(input, weight)
    return out


@scope
def classifier_head(input, num_classes):
    out = adaptive_avg_pooling(input)
    out = linear(out, num_classes)
    return out


@search_space
def cnn_search_space(name, input, max_blocks=3, max_channels=64, num_classes=10):
    out_channels = IntScalarParameter(16, max_channels, step_size=8, name="out_channels")
    kernel_size = CategoricalParameter([3, 5, 7, 9], name="kernel_size")
    stride = CategoricalParameter([1, 2], name="stride")
    num_blocks = IntScalarParameter(1, max_blocks, name="num_blocks")
    exits = []
    out = input
    for i in range(num_blocks.max + 1):
        out = conv_relu(out,
                        stride=stride.new(),
                        out_channels=out_channels.new(),
                        kernel_size=kernel_size.new())
        exits.append(out)

    out = dynamic_depth(*exits, switch=num_blocks)
    out = classifier_head(out, num_classes=num_classes)
    return out

Overwriting space.py


Now we create a config file for `hydra`:

In [2]:
from omegaconf import OmegaConf
import os

cfg_dict ={"_target_": "space.cnn_search_space",
           "name": "cnn_search_space",
           "num_classes": 10,
           "max_channels": 64}

cfg = OmegaConf.create(cfg_dict)

if not os.path.exists("model"):
    os.mkdir("model")
OmegaConf.save(config=cfg, f="model/cnn_search_space.yaml")


### Define Hardware Description

### Configure NAS

### Run NAS

Now we can run the NAS in the "terminal". Note, that we have to append the current working directory to the PYTHONPATH such 
that the search space located in our current directory can be found. If the model file where to be located in the `hannah/models/`
directory that would not be necessary.

In [4]:
%%bash
export PYTHONPATH=$PYTHONPATH:.
hannah-train nas=random_nas model=cnn_search_space nas.n_jobs=1 trainer.max_epochs=5


 **                                            **
/**                                           /**
/**       ******   *******  *******   ******  /**
/******  //////** //**///**//**///** //////** /******
/**///**  *******  /**  /** /**  /**  ******* /**///**
/**  /** **////**  /**  /** /**  /** **////** /**  /**
/**  /**//******** ***  /** ***  /**//********/**  /**
//   //  //////// ///   // ///   //  //////// //   //

[2024-09-01 17:01:38,458][hannah.utils.utils][INFO] - Environment info:
[2024-09-01 17:01:40,087][hannah.utils.utils][INFO] -   Number of GPUs: 1
[2024-09-01 17:01:40,087][hannah.utils.utils][INFO] -   CUDA version: 12.1
[2024-09-01 17:01:40,088][hannah.utils.utils][INFO] -   CUDNN version: 8902
[2024-09-01 17:01:40,088][hannah.utils.utils][INFO] -   Kernel: 5.15.153.1-microsoft-standard-WSL2
[2024-09-01 17:01:40,089][hannah.utils.utils][INFO] -   Python: 3.10.12 (main, Jul 29 2024, 16:56:48) [GCC 11.4.0]
[2024-09-01 17:01:40,089][hannah.utils.utils][INFO] -   PyTorch:

INFO: Seed set to 1234


Running training: 0
[2024-09-01 17:01:46,092][lightning.fabric.utilities.seed][INFO] - Seed set to 1234
[2024-09-01 17:01:46,094][hannah.callbacks.optimization][INFO] - Monitoring the following values for optimization
[2024-09-01 17:01:46,094][hannah.callbacks.optimization][INFO] -   - val_error direction: minimize(1)
[2024-09-01 17:01:46,248][pytorch_lightning.utilities.rank_zero][INFO] - GPU available: True (cuda), used: True
[2024-09-01 17:01:46,248][pytorch_lightning.utilities.rank_zero][INFO] - TPU available: False, using: 0 TPU cores
[2024-09-01 17:01:46,248][pytorch_lightning.utilities.rank_zero][INFO] - IPU available: False, using: 0 IPUs
[2024-09-01 17:01:46,248][pytorch_lightning.utilities.rank_zero][INFO] - HPU available: False, using: 0 HPUs
[2024-09-01 17:01:46,249][pytorch_lightning.utilities.rank_zero][INFO] - `Trainer(limit_train_batches=1.0)` was configured so 100% of the batches per epoch will be used..
[2024-09-01 17:01:46,250][pytorch_lightning.utilities.rank_zero][

INFO: Seed set to 1234


Epoch 4: 100%|██████████| 421/421 [00:19<00:00, 22.10it/s, v_num=0, train_loss=0.0419, val_loss=0.0618]
[2024-09-01 17:03:26,815][lightning.fabric.utilities.seed][INFO] - Seed set to 1234
[2024-09-01 17:03:26,818][pytorch_lightning.utilities.rank_zero][INFO] - Restoring states from the checkpoint path at /home/moritz/projects/hannah/tutorials/automlschool24/trained_models/test/cnn_search_space/0/checkpoints/epoch=4-step=2105.ckpt
[2024-09-01 17:03:26,828][pytorch_lightning.accelerators.cuda][INFO] - LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
[2024-09-01 17:03:26,852][pytorch_lightning.utilities.rank_zero][INFO] - Loaded model weights from the checkpoint at /home/moritz/projects/hannah/tutorials/automlschool24/trained_models/test/cnn_search_space/0/checkpoints/epoch=4-step=2105.ckpt
[2024-09-01 17:03:26,853][py.warnings][WARNING] - /home/moritz/.cache/pypoetry/virtualenvs/hannah-rvIoGOA8-py3.10/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The '

...

Epoch 4: 100%|██████████| 421/421 [00:20<00:00, 20.40it/s, v_num=0, train_loss=0.0942, val_loss=0.0712]
[2024-09-01 17:08:40,333][lightning.fabric.utilities.seed][INFO] - Seed set to 1234
[2024-09-01 17:08:40,335][pytorch_lightning.utilities.rank_zero][INFO] - Restoring states from the checkpoint path at /home/moritz/projects/hannah/tutorials/automlschool24/trained_models/test/cnn_search_space/0/checkpoints/epoch=4-step=2105.ckpt
[2024-09-01 17:08:40,344][pytorch_lightning.accelerators.cuda][INFO] - LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
[2024-09-01 17:08:40,366][pytorch_lightning.utilities.rank_zero][INFO] - Loaded model weights from the checkpoint at /home/moritz/projects/hannah/tutorials/automlschool24/trained_models/test/cnn_search_space/0/checkpoints/epoch=4-step=2105.ckpt
[2024-09-01 17:08:40,367][py.warnings][WARNING] - /home/moritz/.cache/pypoetry/virtualenvs/hannah-rvIoGOA8-py3.10/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The '

INFO: Seed set to 1234


Running training: 0
[2024-09-01 17:08:41,943][lightning.fabric.utilities.seed][INFO] - Seed set to 1234
[2024-09-01 17:08:41,945][hannah.callbacks.optimization][INFO] - Monitoring the following values for optimization
[2024-09-01 17:08:41,945][hannah.callbacks.optimization][INFO] -   - val_error direction: minimize(1)
[2024-09-01 17:08:41,982][pytorch_lightning.utilities.rank_zero][INFO] - GPU available: True (cuda), used: True
[2024-09-01 17:08:41,982][pytorch_lightning.utilities.rank_zero][INFO] - TPU available: False, using: 0 TPU cores
[2024-09-01 17:08:41,982][pytorch_lightning.utilities.rank_zero][INFO] - IPU available: False, using: 0 IPUs
[2024-09-01 17:08:41,982][pytorch_lightning.utilities.rank_zero][INFO] - HPU available: False, using: 0 HPUs
[2024-09-01 17:08:41,982][pytorch_lightning.utilities.rank_zero][INFO] - `Trainer(limit_train_batches=1.0)` was configured so 100% of the batches per epoch will be used..
[2024-09-01 17:08:41,982][pytorch_lightning.utilities.rank_zero][

INFO: Seed set to 1234


Epoch 4: 100%|██████████| 421/421 [00:20<00:00, 20.73it/s, v_num=0, train_loss=0.0759, val_loss=0.060]
[2024-09-01 17:10:17,770][lightning.fabric.utilities.seed][INFO] - Seed set to 1234
[2024-09-01 17:10:17,774][pytorch_lightning.utilities.rank_zero][INFO] - Restoring states from the checkpoint path at /home/moritz/projects/hannah/tutorials/automlschool24/trained_models/test/cnn_search_space/0/checkpoints/epoch=4-step=2105.ckpt
[2024-09-01 17:10:17,783][pytorch_lightning.accelerators.cuda][INFO] - LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
[2024-09-01 17:10:17,800][pytorch_lightning.utilities.rank_zero][INFO] - Loaded model weights from the checkpoint at /home/moritz/projects/hannah/tutorials/automlschool24/trained_models/test/cnn_search_space/0/checkpoints/epoch=4-step=2105.ckpt
[2024-09-01 17:10:17,802][py.warnings][WARNING] - /home/moritz/.cache/pypoetry/virtualenvs/hannah-rvIoGOA8-py3.10/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'v

### Evaluate

In [5]:
!hannah-nas-eval force=True

[2024-09-01 17:11:17,672][nas_eval][INFO] - Current working directory /home/moritz/projects/hannah/tutorials/automlschool24
[2024-09-01 17:11:17,672][nas_eval.prepare][INFO] - Extracting design points
[2024-09-01 17:11:17,672][nas_eval.prepare][INFO] -   Extracting design points for task: nas
[{'cnn_search_space_0.ChoiceOp_0.num_blocks': 3,
  'cnn_search_space_0.conv_relu_0.Conv2d_0.stride': 2,
  'cnn_search_space_0.conv_relu_0.Conv2d_0.weight.kernel_size': 3,
  'cnn_search_space_0.conv_relu_0.Conv2d_0.weight.out_channels': 64,
  'cnn_search_space_0.conv_relu_1.Conv2d_0.stride': 1,
  'cnn_search_space_0.conv_relu_1.Conv2d_0.weight.kernel_size': 9,
  'cnn_search_space_0.conv_relu_1.Conv2d_0.weight.out_channels': 56,
  'cnn_search_space_0.conv_relu_2.Conv2d_0.stride': 2,
  'cnn_search_space_0.conv_relu_2.Conv2d_0.weight.kernel_size': 3,
  'cnn_search_space_0.conv_relu_2.Conv2d_0.weight.out_channels': 24,
  'cnn_search_space_0.conv_relu_3.Conv2d_0.stride': 2,
  'cnn_search_space_0.conv_re

![](comparison_accuracy_macs.png)

![](comparison_accuracy_memory.png)